# Test Convection Diffusion Model


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import torch
import numpy as np
from matplotlib import pyplot as plt

In [ ]:

from src.utils.logger import Logging
from src.data.diffusion_dataset import u, r
from src.nn.pde import diffusion_operator

In [ ]:
TANH_NORM_DIST = "./model/a1/diffusion_tanh_normal_a1.pth"
BSPLINE_DIST = "./model/a1/diffusion_bspline_a1.pth"
CHEBTSHEV_DIST = "./model/a1/diffusion_chebyshev_a1.pth"
GRBF_DIST = "./model/a1/diffusion_grbf_a1.pth"
JACOBI_DIST = "./model/a1/diffusion_jacobi_a1.pth"
TANH_PARAM_DIST = "./model/a2/diffusion_tanh_trainable_a2.pth"
FOURIER_DIST = "./model/a1/diffusion_fourier_a1.pth"


MODEL_PATH_LIST = {
    "tanh": TANH_NORM_DIST,
    "param_tanh": TANH_PARAM_DIST,
    "grbf": GRBF_DIST,
    "bspline": BSPLINE_DIST,
    "chebyshev": CHEBTSHEV_DIST,
    "jacobi": JACOBI_DIST,
    "fourier": FOURIER_DIST,
}

SOLVER_TO_MODULE = {
    "grbf": "src.nn.grbf",
    "jacobi": "src.nn.jacobi",
    "bspline": "src.nn.bspline",
    "chebyshev": "src.nn.chebyshev",
    "MLP": "src.nn.tanh_parameterized",
    "tanh": "src.nn.tanh",
    "fourier": "src.nn.fourier",
}

RESULT = "result/diffusion"

In [9]:
TEST_CHECKPOINT_PATH = os.path.join(PROJECT_ROOT, "result/diffusion")
logger = Logging(TEST_CHECKPOINT_PATH)
model_dirname = logger.get_output_dir()
logger.print(f"Result directory {model_dirname}")

INFO:src.utils.logger:Result directory /home/ubuntu/afrah/code/pinn_learnable_activation/result/diffusion/2024-10-05_18-39-58-494142


## Generate Testing Dataset


In [10]:
num_points = 20

dom_coords = torch.tensor(
    [[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]], dtype=torch.float32, device="cpu"
)

time_ = (
    torch.linspace(dom_coords[0, 0], dom_coords[1, 0], num_points)
    .to("cpu")
    .unsqueeze(1)
    .to(torch.float32)
)
xfa = (
    torch.linspace(dom_coords[0, 1], dom_coords[1, 1], num_points)
    .to("cpu")
    .unsqueeze(1)
    .to(torch.float32)
)

yfa = (
    torch.linspace(dom_coords[0, 2], dom_coords[1, 2], num_points)
    .to("cpu")
    .unsqueeze(1)
    .to(torch.float32)
)

time_, xfa, yfa = torch.meshgrid(time_.squeeze(), xfa.squeeze(), yfa.squeeze())
X_star = torch.hstack(
    (
        time_.flatten().unsqueeze(1),
        xfa.flatten().unsqueeze(1),
        yfa.flatten().unsqueeze(1),
    )
).to("cpu")

## Load the saved models, Test and Print prediction accuracy


In [ ]:
for activation, model_path in MODEL_PATH_LIST.items():
    logger.print(f"MODEL_PATH {model_path}")
    state = torch.load(
        model_path,
    )
    config = state.get("config", {})
    solver = config.get("solver")

    model_activation_name = config.get("activation")
    model_architecture = config.get("network")
    loss_history = state.get("loss_history")
    if solver in SOLVER_TO_MODULE:
        module = __import__(SOLVER_TO_MODULE[solver], fromlist=["PINNKAN"])
        PINNKAN = getattr(module, "PINNKAN")
        model = PINNKAN(model_architecture, model_activation_name).to("cpu")
    else:
        raise ValueError(f"Solver {solver} not found in the dictionary")

    model.load_state_dict(state["model_state_dict"])
    logger.print(f"activation:  , {model_activation_name}")
    logger.print(f"problem: {config.get('problem')}")
    logger.print(f"solver:  , {solver}")
    logger.print(f"network:  , {config.get('network')}")
    logger.print(f"term loss weights :  , {config.get('weights')}")

    u_pred, f_pred = diffusion_operator(
        model, X_star[:, 0:1], X_star[:, 1:2], X_star[:, 2:3]
    )
    if u_pred.is_cuda:
        u_pred = u_pred.cpu()
        f_pred = f_pred.cpu()

    logger.print(
        f"Model {model_activation_name} with iterations: {len(loss_history[next(iter(loss_history))])}"
    )

    u_pred = u_pred.cpu().detach().numpy()
    f_pred = f_pred.cpu().detach().numpy()

    u_analytic = u(X_star).cpu().detach().numpy()
    f_analytic = r(X_star).cpu().detach().numpy()

    error_u = (
        np.linalg.norm(u_analytic - u_pred, 2) / np.linalg.norm(u_analytic, 2)
    ) * 100.0
    error_f = (
        np.linalg.norm(f_analytic - f_pred, 2) / np.linalg.norm(f_analytic + 1e-9, 2)
    ) * 100.0

    logger.print("Relative L2 error_u: {:.2e}".format(error_u))
    logger.print("Relative L2 error_f: {:.2e}".format(error_f))

    for key in loss_history:
        logger.print("Final loss %s: %e" % (key, loss_history[key][-1]))

    logger.print("******************************\n")

logger.print("file directory:", logger.get_output_dir())

INFO:src.utils.logger:MODEL_PATH ../../model/a1/diffusion_tanh_normal_a1.pth
INFO:src.utils.logger:activation:  , None
INFO:src.utils.logger:problem: diffusion
INFO:src.utils.logger:solver:  , tanh
INFO:src.utils.logger:network:  , [3, 50, 50, 50, 1]
INFO:src.utils.logger:term loss weights :  , [10.0, 10.0, 1.0]


INFO:src.utils.logger:Model None with iterations: 60001
INFO:src.utils.logger:Relative L2 error_u: 1.26e+01
INFO:src.utils.logger:Relative L2 error_f: 2.45e+00
INFO:src.utils.logger:Final loss lbcs: 3.028638e-05
INFO:src.utils.logger:Final loss linitial: 5.372631e-05
INFO:src.utils.logger:Final loss lphy: 7.235220e-04
INFO:src.utils.logger:******************************

INFO:src.utils.logger:MODEL_PATH ../../model/a2/diffusion_tanh_trainable_a2.pth
INFO:src.utils.logger:activation:  , tanh
INFO:src.utils.logger:problem: None
INFO:src.utils.logger:solver:  , MLP
INFO:src.utils.logger:network:  , [3, 300, 300, 300, 1]
INFO:src.utils.logger:term loss weights :  , [10.0, 10.0, 1.0]
INFO:src.utils.logger:Model tanh with iterations: 60001
INFO:src.utils.logger:Relative L2 error_u: 1.16e+01
INFO:src.utils.logger:Relative L2 error_f: 7.68e+00
INFO:src.utils.logger:Final loss lbcs: 2.595362e-05
INFO:src.utils.logger:Final loss linitial: 1.190047e-04
INFO:src.utils.logger:Final loss lphy: 5.846